# Minionology: HVSR with seismic nodes
### Skience2023 practical on HVSR, node installation, applications, Geopsy, continuous data analysis

##### Authors:
* Koen Van Noten ([@KoenVanNoten](https://github.com/KoenVanNoten))
* Thomas Lecocq ([@seismotom](https://github.com/ThomasLecocq))
* Martin Zeckra ([@marzeck](https://github.com/marzeck))

##### Introduction:
Geopsy is a powerful software to process ambient noise data. This notebook lists some definitions to read in outputfiles generated by Geopsy.

* .hv files are the Geopsy output files after using the HV module: https://www.geopsy.org/wiki/index.php/H/V_spectral_ratio

##### References:
* Van Noten, K., Devleeschouwer, X., Goffin, C., Meyvis, B., Molron, J., Debacker, T.N. & Lecocq, T. 2022. Brussels’ bedrock paleorelief from borehole-controlled powerlaws linking polarised H/V resonance frequencies and sediment thickness. _Journal of Seismology_ 26, 35-55. DOI: https://doi.org/10.1007/s10950-021-10039-8 pdf: https://publi2-as.oma.be/record/5626/files/2022_VanNotenetal_HVSR_Powerlaw_Brussels.pdf 
* Van Noten, K, Lecocq, Buddha Power, B. (2020). HVSR to Virtual Borehole (1.0). Zenodo. https://doi.org/10.5281/zenodo.4276310
* Zeckra, M., Van Noten, K., Lecocq, T. Submitted. Sensitivity, Accuracy and Limits of the Lightweight Three-Component SmartSolo Geophone Sensor (5 Hz) for Seismological Applications. _Seismica_. Preprint on: https://doi.org/10.31223/X5F073

## Modules needed

In [1]:
import pandas as pd
import numpy as np
import obspy
import os
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.gridspec as gridspec
from obspy.imaging.scripts.scan import Scanner
from obspy.clients.nrl import NRL
from scipy.interpolate import interp1d

## Functions used for reading and plotting Geopsy .hv data

In [2]:
def read_HV(HV_file):
    """
    Read the .hv output files of Geopsy exported from the HV module.
    The definition returns a pandas series of: 
    Freq, A, A_min, A_max 
    with 
    Freq: H/V geometrically averaged over all individual H/V curves (windows)
    A0: the H/V amplitude curve
    A_min: The H/V amplitude minimum standard deviation.
    A_max: The H/V amplitude maximum standard deviation.
    """
    df = pd.read_csv(HV_file, delimiter='\t',names=['Frequency', 'Average', 'Min', 'Max'], comment='#')
    Freq = df["Frequency"]
    A_min = df['Min']
    A = df['Average']
    A_max = df['Max']
    NaNs = np.isnan(df)
    df[NaNs] = 0
    return Freq, A, A_min, A_max

def get_params_from_HV_curve(HV_file):   
    """
    Find the maximum resonance frequency (f0) and corresponding H/V amplitude (A0) of the H/V curve. 
    The function returns the resonance frequency and its amplitude and stvd around f0 as 
    f0_curve, A0_curve, A0_min_curve, A0_max_curve
    Alternatively you also can load this from the .hv file.
    """
    Freq, A, A_min, A_max = read_HV(HV_file)
    A0_curve = np.max(A)
    f0_curve = Freq[np.argmax(A)]
    A0_min_curve = np.max(A_min)
    A0_max_curve = np.max(A_max)
    return f0_curve, A0_curve, A0_min_curve, A0_max_curve

def get_params_from_partial_HV_curve(HV_file, range_min , range_max):   
    """
    Find the maximum resonance frequency (f0) and corresponding H/V amplitude (A0) of parts of the H/V curve given in a list of ranges.
    range_min and range_max gives the range between the maximum needs to be found.
    Default values are 0.2 Hz and 50 Hz, e.g. the default Geopsy values.
    The function returns the resonance frequency and its amplitude and stvd around f0 as 
    f0_curve, A0_curve, A0_min_curve, A0_max_curve
    """
    Freq, A, A_min, A_max = read_HV(HV_file)
    
    #get the index of all the freq. values in specified zone
    position = np.where((Freq>=range_min)&(Freq<=range_max))
    
    #get the freq. and amplitude values that corresponds with the index
    Freqs = Freq[position[0]]
    As = A[position[0]]
    A_mins = A_min[position[0]]
    A_maxs = A_max[position[0]]
    
    #find max. value index
    index_0 = position[0][0]
    maxx = np.argmax(As)
    
    A_range = np.max(As)
    A_min_range = np.max(A_mins)
    A_max_range = np.max(A_maxs)
    f_range = Freqs[index_0 + maxx]  

    return f_range, A_range, A_min_range, A_max_range

def findf0A0InRanges(freq_ranges,frequency,amplitude):
    #description: find the maximum amplitude and corresponding frequency value in every range zone
    #input: 
    #freq_ranges: defined frequency zones to find max. in
    #frequency: freq. values (can be interpolated or non-interpolated values)
    #amplitude: amplitude values (can be interpolated or non-interpolated values)
    #return: list of max. amplitude values (A0) and corresponding freq. values (f0)
    
    f0 = []
    A0 = []
    for i in range(0,len(freq_ranges)):
        #get the index of all the freq. values in defined zone
        position = np.where((frequency>=freq_ranges[i][0])&(frequency<=freq_ranges[i][1]))
        #get the freq. and amplitude values that corresponds with the indix
        Freq_range = frequency[position]
        HVmean_range = amplitude[position]
        #find max. values 
        maxx = np.argmax(HVmean_range)
        HVmean_max = round(np.max(HVmean_range),3)
        Freq_max = round(Freq_range[maxx],3)

        #store f0 and A0 value to list 
        A0.append(HVmean_max)
        f0.append(Freq_max)

def get_params_from_HV(HV_file):
    """
    Read all the necessary info that is available in a Geopsy .HV file.
    The function returns 
    f0_avg, f0_win, error, A0, nw_avg, nw_win, f_min, f_max
    with
    * f0 avg: scanning the average curve and identifying the frequency at which the maximum amplitude occurs (from GEOPSY)
    * f0 min: f0_win/stddev (from GEOPSY)
    * error: standard deviation on f0 (from GEOPSY)
    * A0: maximum amplitude (from GEOPSY)
    * nw_avg: number of windows from f0_avg(from GEOPSY)
    * nw_win: number of windows (from GEOPSY)
    * f min: minimum f0_win.stddev (from GEOPSY)
    * f max: maximum f0_win.stddev (from GEOPSY)
    
    See Github for more info: https://github.com/KoenVanNoten/HVSR_to_virtual_borehole/blob/master/Get%20f0s%20from%20geopsy%20hv%20files.py
    """
    df = pd.read_csv(HV_file, nrows=5, skiprows=1, header=None) #opening the .hv file
    rows = ["n_avg", "f0_avg", "n_win", "f0s", "peak_amp"] #define rows to write in the output file
    data = {}
    for row in rows:
                data[row] = ""
    delims = ["=", "\t", "=", "\t", "\t"]
    for id2, item in df.iterrows():
                XXX = item[0].split(delims[id2])
                data[rows[id2]] = np.asarray(XXX[1:], dtype=float).flatten()
    data["f0_win"], data["f_min"], data["f_max"] = data["f0s"]
    del data["f0s"]

    f0_avg = data["f0_avg"][0]
    f0_win = data["f0_win"]
    error = data["f0_win"] - data["f_min"]
    f_min =  data["f_min"]
    f_max =  data["f_max"]
    A0 = data["peak_amp"][0]
    nw_avg = data["n_avg"][0]
    nw_win = data["n_win"][0]
    
    return f0_avg, f0_win, error, A0, nw_avg, nw_win, f_min, f_max

def get_interpolated_values_from_HV(HV_file, interpol, f0_win):
    """
    In the default setting, Geopsy only exports 100 frequency-amplitude samples for the computed HVSR curve.
    One can increase this number by:
       - increasing the logaritmic step count in Geopsy manually (max = 9999)
       - by interpolating between the samples and improve the accuracy of picking f0 (this script)
    Increasing the samples in Geopsy to 9999 gives the same results, but one might have forgotten to do this while processing
    so this interpolation offers a nice twist to solve this.
    The part in below executes the interpolation up to 15000 samples (default)
    See paper Van Noten et al. (2022) for more information. Same method is applied in the HVSR_to_virtual_borehole module.
    Interpol: nr of interpolated samples
    f0_win: load the f0 from windows from get_params_from_HV:
    """
    HV_data = np.genfromtxt(HV_file, delimiter='\t', usecols=(0, 1))
    #print(HV_data)
    f_orig = HV_data[:, 0] #original frequency data
    print("nr of samples:", len(f_orig))
    A_orig = HV_data[:, 1] #original amplitude data

    func = interp1d(f_orig, A_orig, 'cubic') #IN
    f0_interpolated = np.linspace(f_orig[0], f_orig[-1], 15000) #interpolation for 15000 samples
    A0_interpolated = func(f0_interpolated) #applying the function for the new Amplitude
    A0_int = np.argmax(A0_interpolated)
    f0_int = f0_interpolated[A0_int]
       
    # With the interpolated data new columns can be calculated to compare the interpolated values and the ones provided by Geopsy
    f0_int_diff = f0_int - f0_win #difference between f0_interpolated and f0_geopsy
    
    return f0_int, A0_int, f0_int_diff


def write_HVline_to_db(db_HVSR, f_min, f0_win, f0_avg, f0_int, f0_int_diff, error, f_max, A0, nw_win):
        """
        Write all params to the HVSR database using a pandas db.loc function.
        
        Before using this function, empty columns should be created by the code below: 
        #### Initializing empty columns that need to be filled from the Geopsy .hv files
        for _ in ["f0_min", "f0_win", "f0_avg", "f0_int", "f0_int_diff", "error", "f0_max", "A0", "nw"]:
            db_HVSR[_] = 0.
        
        Then the value of each colomn will be filled in. 
        """
        db_HVSR.loc[id, "f0_min"] = f_min #f0 min
        print("f0_min:", round(f_min,3), "Hz")
        db_HVSR.loc[id, "f0_win"] = f0_win #average f0 computed by averaging the peak f0 values of all individual windows
        print("f0_win:", round(f0_win,3), "Hz")
        db_HVSR.loc[id, "f0_avg"] = f0_avg #f0 corresponding to the maximum amplitude of the average f0 - Amplitude curve
        print("f0_avg:", round(f0_avg,3), "Hz")
        db_HVSR.loc[id, "f0_int"] = f0_int #interpolated f0 from 15000 samples
        print("f0_int:", round(f0_int,3), "Hz")
        db_HVSR.loc[id, "f0_int_diff"] = f0_int_diff #difference between f0_interpolated and f0_win
        print ("f0_ip_diff:", round(f0_int_diff,3), "Hz")
        db_HVSR.loc[id, "error"] = error #error on f0_win in Geopsy
        print("error:", round(error,3), "Hz")
        db_HVSR.loc[id, "f0_max"] = f_max #f0 max
        print("f0_max:", round(f_max,3), "Hz")
        db_HVSR.loc[id, "A0"] = A0 #A0
        print("A0:", round(A0,3))
        db_HVSR.loc[id, "nw"] = nw_win #number of windows used to compute f0
        print("nw:", int(nw_win), "windows")
        print('')
        
        return db_HVSR
    
def get_paramString():
    """
    Creation of an empty ParamString that can be filled in with optional parameters. 
    This creates a .log file, similar as when Geopsy is used manually
    """
    ## We create a default PARAM multiline string with the static components for whole processing 
    ## and formatters for variables
    paramsString = '''PARAMETERS_VERSION=1    \nFROM_TIME_TYPE=Absolute\nFROM_TIME_TEXT={tStart}\nTO_TIME_TYPE=Absolute\nTO_TIME_TEXT={tEnd}\nREFERENCE=\nCOMMON_TIME_WINDOWS=false\nWINDOW_LENGTH_TYPE=Exactly\nWINDOW_MIN_LENGTH(s)={winLen}\nWINDOW_MAX_LENGTH(s)={winLen}\nWINDOW_MAX_COUNT=0\nWINDOW_MAXIMUM_PRIME_FACTOR=11\nBAD_SAMPLE_TOLERANCE (s)=0\nBAD_SAMPLE_GAP (s)=0\nWINDOW_OVERLAP (%)={overlap}\nBAD_SAMPLE_THRESHOLD_TYPE={threshold}\nBAD_SAMPLE_THRESHOLD_VALUE (%)={threshold_pct}\nANTI-TRIGGERING_ON_RAW_SIGNAL (y/n)=n\nANTI-TRIGGERING_ON_FILTERED_SIGNAL (y/n)=n\nSEISMIC_EVENT_TRIGGER (y/n)=n\nSEISMIC_EVENT_DELAY (s)=-0.1\nWINDOW_TYPE=Tukey\nWINDOW_REVERSED=n\nWINDOW_ALPHA=0.1\nSMOOTHING_METHOD=Function\nSMOOTHING_WIDTH_TYPE=Log\nSMOOTHING_WIDTH={KO}\nSMOOTHING_SCALE_TYPE=Log\nSMOOTHING_WINDOW_TYPE=KonnoOhmachi\nSMOOTHING_WINDOW_REVERSED=n\nMINIMUM_FREQUENCY={minFreq}\nMAXIMUM_FREQUENCY={maxFreq}\nSCALE_TYPE_FREQUENCY=Log\nSTEP_TYPE_FREQUENCY=Count\nSAMPLES_NUMBER_FREQUENCY=500\n#STEP_FREQUENCY=1.00231\nHIGH_PASS_FREQUENCY=0\nHORIZONTAL_COMPONENTS={horizontals}\nHORIZONTAL_AZIMUTH={azimuth}\nROTATION_STEP={rotSteps}\nFREQUENCY_WINDOW_REJECTION_MINIMUM_FREQUENCY={rej_min_freq}\nFREQUENCY_WINDOW_REJECTION_MAXIMUM_FREQUENCY={rej_max_freq}\nFREQUENCY_WINDOW_REJECTION_STDDEV_FACTOR={rej_stdev}\nFREQUENCY_WINDOW_REJECTION_MAXIMUM_ITERATIONS={rej_it}\n'''
    return paramsString

## Definitions used for plotting Geopsy .hv data

In [3]:
## Plot the HV curve using all data loaded above
## standard deviation is a fill_betweenx
def plot_HV(Freq, A, A_min, A_max, f0, A0, Amin, Amax, analysis, color):
    
    """
    Plot the HV curve from an .hv log file using all necessary params:
    Freq, A, A_min, A_max, Amax_f0, Fmax_f0, Amin, Amax, label, color
    Label: fill in whatever you want
    color: color the H/V curve
    Other params see get_params_from_HV() and get_params_from_HV_curve()
    """
    
    plt.plot(A, Freq, c=color, label='$f_0$ %s'%analysis, zorder = 0)
    plt.fill_betweenx(Freq, A_min,A_max,facecolor='silver', edgecolor="k", alpha=0.3, label = 'error')
    print('%s: At %s Hz (f0), the maximum H/V amplitude is %s ± %s'%(analysis, 
                                                                     round(f0,2), 
                                                                     round(A0,1), 
                                                                     round(Amax-A0,1)))

## Compute the depth using the Brussels powerlaw of Van Noten et al. (2022): DOI: https://doi.org/10.1007/s10950-021-10039-8
## print the bedrock depth from the f0 value
def get_Brussels_powerlaw(f0):
    h = 88.631 * np.power(f0,-1.683)
    print('    Bedrock at %.2f m depth'%h)

### colorline - need for plotting a colored line along the HVSR profile
def colorline(x, y, z, cmap=plt.get_cmap('copper'), linewidth=10, alpha=1.0):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """
    z = np.asarray(z)
    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, linewidth=linewidth, alpha=alpha)

    ax = plt.gca()
    ax.add_collection(lc)

    return lc


### segments - need for plotting a colored line along the HVSR profile
def make_segments(x, y):
    """"
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    """
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    return segments


## script to plot a virtual borehole from an HV file
def HV_to_virtual_borehole(HV_file,ID, Z):
    """
    
    """
    
    ### load HV data
    f, A, A_min, A_max = read_HV(HV_file)
    
    ### Instead of using the output of Geopsy, one can interpolate the entire HVSR curve for 15000 points to improve f0
    ### Check get_interpolated_values_from_HV? for info 
    
    if interpolate:
        # interpolate for A0
        func = interp1d(f, A, 'cubic')
        f_ip = np.linspace(f[0], f[len(f) - 1], 15000)
        A_ip = func(f_ip)

        # interpolate for A_min
        func = interp1d(f, A_min, 'cubic')
        A_min_ip = func(f_ip)

        # interpolate for A_max
        func = interp1d(f, A_max, 'cubic')
        A_max_ip = func(f_ip)

        # overwrite original f, A
        A = A_ip
        f = f_ip
        A_min = A_min_ip
        A_max = A_max_ip

    A_plot = A * 0

    ### Plot the Amplitude - frequency diagram and the virtual borehole
    gs = gridspec.GridSpec(1, 2, width_ratios=[12, 1])
    plt.suptitle(ID)
    ax0 = plt.subplot(gs[0])
    plt.plot(A,f, linewidth=0.7)
    maxx = np.argmax(A)
    print("A0:", round(A[maxx],2), "fmax: ",round(f[maxx],2))
    plt.fill_betweenx(f, A_min, A_max, color = 'lightgrey', zorder = -100)
    ax0.set_yscale('log')
    colorline(A_plot, f, A, cmap='viridis', linewidth=5)
    colorbar = colorline(A_plot, f, A, cmap='viridis', linewidth=10)
    plt.colorbar(colorbar, label = "Amplitude")


    #### function to find and plot f0 values from the geopsy files
    #A_max = np.max(A0) # find largest amplitude in the geopsy or interpolated values
    #A_max = np.where(np.max(A)[0.8,1.2])   # sometimes amplitude is higher at other values dan f0, avoid this by defining a range in which we need to search                           
    
    ### get the HV info from the HV file
    f0_avg, f0_win, error, A0_geopsy, nw_avg, nw_win, f_min, f_max = get_params_from_HV(HV_file)
 
    if interpolate:
        ### plot a horizontal line for the average if you want to plot the interpolated f0 value
        plt.axhline(y=f[maxx], xmin=0, xmax=20, color='red', linewidth=0.5, zorder=-100)
        print("f0_ip = ", round(f[maxx], 3))
        f0_min = float(f[maxx] - error)
        f0_max = float(f[maxx] + error)
        
        # convert frequency to depth using a powerlaw relation
        if depth_conversion == 'powerlaw':
            depth = a_pw * np.power(f[maxx], b_pw)
        # convert frequency to depth a fixed Vs
        if depth_conversion == 'Vs':
            depth = Vs / (f[maxx] * 4)
            
    else:
        ### plot a horizontal line for the average if you want to plot the geopsy f0 value
        ### still discussion needed if f0_avg or f0_win is the best? fo_ip = f0_avg
        plt.axhline(y=f0_avg, xmin=0, xmax=20,color = 'red', linewidth=0.5, zorder = -100)
        f0_min = float(f0_avg - error)
        f0_max = float(f0_avg + error)
        
        # convert frequency to depth using a powerlaw relation
        if depth_conversion == 'powerlaw':
            depth = a_pw * np.power(f0_avg, b_pw)
            
        # convert frequency to depth a fixed Vs
        if depth_conversion == 'Vs':
            depth = Vs / (f0_avg * 4)
        
    ### plot horizontal lines for f0_min and f0_max using the error provided by geopsy
    plt.axhline(y=f0_min, xmin=0, xmax=20, color='grey', linewidth=0.5, ls='--', zorder=-100)
    plt.axhline(y=f0_max, xmin=0, xmax=20,color = 'grey', linewidth=0.5, ls = '--', zorder = -100)
    plt.title("$f_0$ int.: %.3f" % f[maxx] + r"$\pm$%.3f" %error + "(err)" + "; $A_0$: %.2f" % A0_geopsy, size=10)
    plt.ylabel("Frequency (Hz)", fontsize=10)
    plt.xlabel("Amplitude", fontsize=10)
    if auto_amplitude:
        plt.xlim(-1, np.max(A_max))
    else:
        plt.xlim(-1,manual_amplitude)
    plt.ylim(freq[0],freq[1])
  
    #### Making the virtual borehole in function of depth
    ax1 = plt.subplot(gs[1])
        
    if depth_conversion == 'powerlaw':    
        # convert frequency to depth using the powerlaw relation for all frequencies
        h = a_pw*np.power(f,b_pw)

        #defining the errorbars in the virtual borehole
        depth_min = a_pw*np.power(f0_min,b_pw)
        depth_max = a_pw*np.power(f0_max,b_pw)

    if depth_conversion == 'Vs':
        h = Vs / (f * 4)
        
        #defining the errorbars in the virtual borehole
        depth_min =  Vs / (f0_min * 4)
        depth_max = Vs / (f0_max * 4)

    # calculating absolute depth; Z = altitude of borehole
    bedrock = Z - depth
    bedrock_min = Z - depth_min
    bedrock_max = Z - depth_max
    all_depths = Z - h
    print ("Z: ", round(Z,2))
    print ("bedrock at", round(bedrock,1), " m (range: ", round(bedrock_min,1), "m, ", round(bedrock_max,1), "m)")

    colorline(A_plot, all_depths, A, cmap='viridis', linewidth=50)

    plt.ylim(Z+10, Z-depth-20)
    plt.gca().invert_yaxis()
    plt.ylabel("Altitude bedrock (m TAW)", fontsize=10)
    ax1.axes.get_xaxis().set_ticks([])
    ax1.yaxis.set_label_position("right")
    ax1.yaxis.tick_right()
    plt.axhline(y=bedrock, xmin=0, xmax=20,color = 'red', linewidth=0.8, zorder = 100)
    plt.axhline(y=bedrock_min, xmin=0, xmax=20,color = 'black', linewidth=0.8, zorder = 100)
    plt.axhline(y=bedrock_max, xmin=0, xmax=20,color = 'black', linewidth=0.8, zorder = 100)
    plt.axhline(y=Z, xmin=0, xmax=20,color = 'black', linewidth=0.7, zorder = 100)
    plt.annotate('%s'%int(Z) + " m", xy=(0.,Z), fontsize = 8)
    plt.title("Bedrock at %.0f" % (bedrock) + " m", size=10)
    
## script to plot polarisation data
def plot_polarisation_data(in_filespec,ID, limfreq_min,limfreq_max, A0_max):
    """
    The Geopsy output is not intuitive as polar data are plotted in an X (Frequency) - Y (Azimuth) diagram 
    instead of a 360° diagram. This script loads a Geopsy HV rotate module .grid file and 
    replots it into a more understandable polar plot. It will search the azimuth at which the 
    maximum resonance frequency occurs. 

    Data needed:
    * .hv.grid file
    * ID (node) name
    * limfreq_min,limfreq_max: frequency range between the polar plot is made
    * A0_max = maximum HV amplitude (can be given or read from the HVSR database)
    
    Following data is returned:
    A_max: maximum amplitude at resonance frequency deduced from the HVSR polarisation analysis
    max_freq: Resonance frequency at A_max
    max_Azi: Azimuth at which resonance frequency is maximum (deduced from polarisation analysis)
    A_min: minimum amplitude at resonance frequency deduced from the HVSR polarisation analysis
    min_freq: Azimuth at which resonance frequency is minimal (deduced from polarisation analysis)
    min_Azi: Azimuth at which resonance frequency is minimum (deduced from polarisation analysis)
    """
    df = pd.read_csv(in_filespec, delimiter=' ', skiprows=0, engine = 'python')
    freq = df["x"]
    Azi = df["y"]
    A = df["val"]

    ## Get the rotation step. Default = 10° in Geopsy. Can be changed since Geopsy version 3.3.3
    groups = df.groupby(Azi)
    rotation_classes = len(groups) ## gives the amount of rotation step classes. = 19 for 10° steps
    rotation_step = int(180/(rotation_classes-1)) ## gives the rotation_step

    ### find the polarization by searching for maximum amplitude for each azimuth
    Amax = []
    freqmax = []
    Azimax = []

    for i in np.arange(0,180+rotation_step,rotation_step):
        index = np.array(np.where((Azi == i)))[0]

        ## search for maximum and minimum A0 in a given frequency range
        if freq_range:
            index_range = []
            for ind in index:
                if freq[ind] >= f_range[0]:
                    if freq[ind] <= f_range[1]:
                        index_range.append(ind)
            index = index_range

        ## find maximum amplitude of each angle (0--> 180) so we later can find the max and min value in this list
        ## append the max Amplitude in the i angle
        Amax.append(np.max(A[index]))
        ## append the frequency corresponding to that amplitude
        freqmax.append(freq[index[0] + np.argmax(A[index])])
        ## append the angle to a list
        Azimax.append(i)
        ## find the maximum in the max amplitude list
        A_max = np.max(Amax)

    #find the minima and maxima (white and red dots in the plot)
    max_freq = freqmax[np.argmax(Amax)]
    max_Azi = Azimax[np.argmax(Amax)]
    A_min = np.min(Amax)
    min_freq = freqmax[np.argmin(Amax)]
    min_Azi = Azimax[np.argmin(Amax)].transpose()
    
    if plot_fig:
        ####### Let's plot
        plt.figure(figsize=(6.5,5))
        ax = plt.subplot(111, polar=True)
        
        ### reshape the amplitude column
        A_reshape = A.values.reshape(rotation_classes,int(len(freq)/rotation_classes))

        ## define the region where Amplitudes have to be plotted
        ## freq = xi; yi = Azimuth; A_reshape is amplitude
        xi = np.array([np.geomspace(np.min(freq), np.max(freq), int(len(freq)/rotation_classes)),]*rotation_classes)
        yi = np.array([np.arange(0,190,rotation_step),]*int(len(freq)/rotation_classes)).transpose()

        ### flip the polar plot to mirror it on the W side
        yj = np.array([np.arange(180,370,rotation_step),]*int(len(freq)/rotation_classes)).transpose()

        ### for log plots - use fixed amplitudes for whole the dataset or use a flexible A0max for each plot
        if A0_max == 0:
            plt.pcolormesh(np.deg2rad(yi), np.log(xi), A_reshape, shading='auto', 
                           cmap='viridis', vmin=0, vmax=np.round(np.max(A), 0), rasterized=True)
            plt.pcolormesh(np.deg2rad(yj), np.log(xi), A_reshape, shading='auto', 
                           cmap='viridis', vmin=0, vmax=np.round(np.max(A), 0),rasterized=True)
        else:
            plt.pcolormesh(np.deg2rad(yi), np.log(xi), A_reshape, shading='auto', 
                           cmap='viridis', vmin=0, vmax=np.round(A0_max, 0), rasterized=True)
            plt.pcolormesh(np.deg2rad(yj), np.log(xi), A_reshape, shading='auto', 
                           cmap='viridis', vmin=0, vmax=np.round(A0_max, 0), rasterized=True)

        cbar = plt.colorbar(pad = 0.1, format = '%.0f')
        cbar.set_label('H/V Amplitude', rotation=90)

        if A_max < 10:
            format_max = round(A_max,3)
        else:
            format_max = round(A_max,2)
        if A_min < 10:
            format_min = round(A_min, 3)
        else:
            format_min = round(A_min,2)

        ### plot the min and maxima (red and white dots)
        plt.scatter(np.deg2rad(max_Azi), np.log(max_freq), c='red', edgecolor='black',
                    label = "Max. Ampl. ("+ str(format_max) + ') at \n' + str(max_Azi) + '° - '
                            + str(max_Azi+180) +'° for $f_0$ ' + format(round(max_freq,2), '.2f') + 'Hz', zorder = 3)
        plt.scatter(np.deg2rad(min_Azi), np.log(min_freq), c='white', edgecolor='black',
                    label = "Min. Ampl. ("+ str(format_min) + ') at \n' + str(min_Azi) + '° - ' + str(min_Azi+180) +'° for $f_0$ ' + format(round(min_freq,2), '.2f') + 'Hz', zorder = 3)
        plt.scatter(np.deg2rad(max_Azi+180), np.log(max_freq), c='red', edgecolor='black', zorder = 3)
        plt.scatter(np.deg2rad(min_Azi+180), np.log(min_freq),c='white', edgecolor='black', zorder = 3)

        ### modify the rotational options
        ax.set_theta_direction('clockwise')
        ax.set_theta_zero_location('N')
        ax.set_rlabel_position(0)
        ax.text(np.radians(180),np.log(ax.get_rmax()/3.5),'Frequency',fontsize=8,
                rotation=90,ha='left',va='center', color= 'white')

        # limits of the frequency and modify the ticks of the frequency
        if auto_freq:
            limfreq_min = round(max_freq,1) - 0.4
            limfreq_max = round(max_freq,1) + 0.4

        ax.set_rlim(np.log(limfreq_min),np.log(limfreq_max))
        pos_list = np.log(np.arange(limfreq_min+0.1,limfreq_max,steps/2))
        freq_list = np.round(np.arange(limfreq_min+0.1,limfreq_max,steps),3)
        freqs = []
        for i in freq_list:
            freqs.append(i)
            freqs.append('')

        ax.yaxis.set_major_locator(ticker.FixedLocator(pos_list))
        ax.yaxis.set_minor_locator(ticker.FixedLocator(pos_list+0.1))
        ax.yaxis.set_major_formatter(ticker.FixedFormatter((freqs)))

        rlabels = ax.get_ymajorticklabels()
        for label in rlabels:
            label.set_color('white')

        # Specify the ticks of the azimuth
        ax.set_xticks(np.pi/180. * np.linspace(0,  360, 18, endpoint=False))
        ax.yaxis.set_tick_params(labelsize=9)

        plt.legend(loc='best', bbox_to_anchor=(-0.4, -0.35, 0.5, 0.5), frameon=False)
        plt.grid(linestyle='-.', linewidth=0.2, alpha = 1, zorder = 200, color = 'grey')

        # Plot the title
        plt.title("Resonance frequency polarisation of %s"%ID, y=1.08)
        plt.tight_layout()
    if save_fig:
        plt.savefig(os.path.join(out_folder, '%s'%ID + '_polarisation.png'))
        
    return A_max, max_freq, max_Azi,A_min, min_freq, min_Azi
    

### Inventory of IGU-16 HR 3C

In [4]:
def create_IGU_16_HR3C_inv(network, network_code, node_nr, station_lat, station_lon, elevation, site_name, 
    channel_code, channel_loc, sample_rate):

    inv = Inventory(
        # We'll add networks later.
        networks=[],
        source="Seismology.be")

    net = Network(
        # This is the network code according to the SEED standard - BE_ for nodes
        code=network_code,
        # A list of stations. We'll add one later.
        stations=[],
        description="SmartSolo 3C IGU16HR nodes - Seismology.be",
        # Start-and end dates when nodes were bought
        start_date=obspy.UTCDateTime(2021, 6, 1))

    sta = Station(
        # This is the station code according to the SEED standard.
        code=node_nr,
        latitude=station_lat,
        longitude=station_lon,
        elevation=elevation,
        site=Site(name=site_name))
    
    cha = Channel(
        # This is the channel code according to the SEED standard. - # (HH)(DP(Z)(N)(E) for nodes
        code=channel_code,
        # This is the location code according to the SEED standard.
        location_code=channel_loc,
        latitude=station_lat,
        longitude=station_lon,
        elevation=elevation,
        depth=0.0,
        azimuth=0.0,
        dip=-90.0,
        sample_rate=sample_rate)

    # By default this accesses the NRL online. Offline copies of the NRL can
    # also be used instead
    nrl = NRL()

    # load response from nrl
    response_node = nrl.get_response(
        datalogger_keys=['DTCC (manufacturers of SmartSolo','SmartSolo IGU-16', '36 dB (64)', '250', 'Linear Phase', 'Off'],
        sensor_keys=['DTCC (manuafacturers of SmartSolo)', '5 Hz', 'Rc=1850, Rs=430000'])
    # correct typo in gain
    response_node.response_stages[0].stage_gain = 76.7
    # add additional gain stage for node digitizer
    response_node.response_stages[1].stage_gain *= 3355.4428
    response_node.recalculate_overall_sensitivity()

    # Now tie it all together.
    cha.response = response_node
    sta.channels.append(cha)
    net.stations.append(sta)
    inv.networks.append(net)
    print(sta.code)

    return inv

In [ ]:
'''
### Get node response:
start_date
inv = create_inv("","BE", df_loc.loc[node].node_channels,
                     df_loc.loc[node].lat, df_loc.loc[node].lon,
                     0, df_loc.loc[node].Station,st1[0].stats.channel, "", 250)
'''